In [3]:
import numpy as np
from collections import Counter
from model import Board, RandomPlayer, MiniMaxPlayer

In [4]:
results = []

for game in range(50):
    
    board = Board(state=np.zeros((3,3), dtype=int))
    player1 = RandomPlayer('X')
    player2 = MiniMaxPlayer('O', search_depth=4)

    turns = 0
    active_player = player1
    inactive_player = player2
    while not board.is_finished(board.state):
        move = active_player.move(board)
        board.update_board(move=move, marker=active_player.marker)
        turns += 1
        # switch players
        active_player, inactive_player = inactive_player, active_player

#     Display game result
#     board.draw(board.state)
#     print(f"The winner is: {board.is_finished(board.state)}")

    if game % 10 == 0:
        print(f"{game} games completed.")
    results.append(board.is_finished(board.state))

0 games completed.
10 games completed.
20 games completed.
30 games completed.
40 games completed.


In [5]:
Counter(results)

Counter({'O': 35, 'draw': 11, 'X': 4})